In [1]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


C:\Users\Polad_Babashev\AppData\Local\Continuum\anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\Polad_Babashev\AppData\Local\Continuum\anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [10]:
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
import spacy
import warnings
import utils
import os
import pickle

from stop_words import get_stop_words
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from tqdm import tqdm_notebook as tqdm
from pymystem3 import Mystem

warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
stop_words = get_stop_words('ru')
stop_words.extend(['искусственный интеллект', 'машинное обучение', 'ии'])

In [ ]:
'свой' in stop_words

In [3]:
df = utils.read_data(['data/meduza.csv', 'data/tjornal.csv', 'data/nplus.csv'], **{'parse_dates': ['article_time']})

In [4]:
df.shape

(1181, 4)

In [5]:
df.dtypes

article_url                object
article_time       datetime64[ns]
article_head               object
article_content            object
dtype: object

## Preprocessing

In [6]:
lemmatizer = Mystem()

In [7]:
texts_content = df.article_content.values.tolist()

In [8]:
%%time
texts_content = [simple_preprocess(text, deacc=True, min_len=2, max_len=30) for text in texts_content]

Wall time: 4.94 s


In [9]:
%%time
print('lemmatization...')
texts_content = utils.lemmatization(texts_content, lemmatizer)

lemmatization...


100%|██████████████████████████████████████████████████████████████████████████████████| 1181/1181 [25:16<00:00,  1.28s/it]


Wall time: 25min 16s


In [11]:
texts_content = pickle.load(open('lemmatized.pkl', 'rb'))

In [12]:
%%time
texts_content = [[w for w in text if not w == ' '] for text in tqdm(texts_content)]
print('removing stopwords...')
texts_content = utils.remove_stopwords(texts_content, stop_words)
print('bigrams...')
bigram = gensim.models.Phrases(texts_content, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
texts_content = utils.make_bigrams(texts_content, bigram_mod)



removing stopwords...
bigrams...
Wall time: 12.8 s


In [13]:
%%time
id2word = corpora.Dictionary(texts_content)
texts = texts_content
corpus = [id2word.doc2bow(text) for text in texts]

Wall time: 1.47 s


In [14]:
readable_corpus = [[(id2word[id], freq) for id, freq in cp] for cp in corpus]

## Training LDA

### topics = 10

In [15]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                            num_topics=10,
                                           update_every=3,
                                           chunksize=50,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Wall time: 41.4 s


In [16]:
lda_model.print_topics()

[(0,
  '0.013*"система" + 0.008*"технология" + 0.007*"компания" + 0.005*"машина" + 0.005*"данные" + 0.005*"позволять" + 0.005*"работа" + 0.004*"использовать" + 0.004*"работать" + 0.004*"например"'),
 (1,
  '0.010*"данные" + 0.008*"главнивц" + 0.007*"шерлок" + 0.007*"система" + 0.007*"компания" + 0.005*"разработка" + 0.005*"рассказывать" + 0.005*"доступ" + 0.005*"работать" + 0.004*"медуза"'),
 (2,
  '0.008*"приложение" + 0.008*"пользователь" + 0.007*"google" + 0.006*"игра" + 0.005*"свой" + 0.005*"например" + 0.005*"яндекс" + 0.004*"работать" + 0.004*"неиросеть" + 0.004*"понимать"'),
 (3,
  '0.007*"nature" + 0.003*"григорьев" + 0.002*"репетитор" + 0.001*"мирэа" + 0.001*"зачет" + 0.001*"информационноя_безопасность" + 0.001*"rex" + 0.000*"людскои" + 0.000*"яркий" + 0.000*"рубиться"'),
 (4,
  '0.023*"музыка" + 0.017*"музыкальный" + 0.009*"музыкант" + 0.009*"альбом" + 0.008*"произведение" + 0.006*"песня" + 0.005*"композитор" + 0.004*"композиция" + 0.004*"форма" + 0.004*"слушатель"'),
 (5,
  

In [17]:
lda_model.log_perplexity(corpus)

-8.92887093002859

In [18]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts_content, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [19]:
coherence_lda

0.4610680547075945

### topics = 5

In [20]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5,
                                            update_every=3,
                                            chunksize=50,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

Wall time: 28.7 s


In [21]:
lda_model.print_topics()

[(0,
  '0.007*"телефон" + 0.005*"камера" + 0.005*"экран" + 0.005*"приложение" + 0.004*"apple" + 0.004*"компания" + 0.004*"рассказывать" + 0.004*"работать" + 0.004*"свой" + 0.004*"главнивц"'),
 (1,
  '0.014*"неиросеть" + 0.006*"алгоритм" + 0.005*"изображение" + 0.005*"создавать" + 0.005*"\n" + 0.005*"работа" + 0.004*"результат" + 0.004*"фильм" + 0.003*"помощь" + 0.003*"автор"'),
 (2,
  '0.049*"сложность" + 0.049*"коэффициент" + 0.010*"алгоритм" + 0.007*"система" + 0.007*"работа" + 0.005*"использовать" + 0.005*"данные" + 0.005*"неиросеть" + 0.005*"изображение" + 0.005*"помощь"'),
 (3,
  '0.016*"игра" + 0.007*"игрок" + 0.006*"играть" + 0.005*"ия" + 0.004*"го" + 0.004*"deepmind" + 0.004*"свой" + 0.004*"матч" + 0.003*"становиться" + 0.003*"герой"'),
 (4,
  '0.007*"компания" + 0.005*"свой" + 0.005*"система" + 0.004*"технология" + 0.004*"например" + 0.004*"становиться" + 0.004*"работать" + 0.004*"пользователь" + 0.004*"\n" + 0.004*"использовать"')]

In [22]:
lda_model.log_perplexity(corpus)

-8.497914710494909

In [23]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts_content, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [24]:
coherence_lda

0.4363723122746693